In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
from datetime import datetime
from typing import Dict, List, Optional

from urllib.parse import unquote # URL 디코딩 -> ')' 의 경우 %29로 인코딩 되어있어서 href로 가져올 때 디코딩 필요
from urllib.parse import quote # URL 인코딩 -> ' '의 경우 %20으로 인코딩 필요

## K-POP Fandom Wiki API

- Group Crawling

In [3]:
class KpopGroupCrawler_memberDict:
    def __init__(self):
        self.base_url = "https://kpop.fandom.com/api.php"
        self.base_wiki_url = "https://kpop.fandom.com"
    
    def get_group_info(self, group_name: str) -> Dict:
        """
        특정 K-pop 그룹의 정보를 크롤링합니다.
        
        Args:
            group_name (str): 그룹명 (예: "Red_Velvet", "BTS", "BLACKPINK")
        
        Returns:
            Dict: 그룹 정보가 담긴 딕셔너리
        """
        params = {
            "action": "parse",
            "page": group_name,
            "format": "json"
        }
        
        try:
            res = requests.get(self.base_url, params=params)
            res.raise_for_status()
            data = res.json()
            
            if 'parse' not in data:
                print(f"페이지를 찾을 수 없습니다: {group_name}")
                return {}
            
            html = data["parse"]["text"]["*"]
            soup = BeautifulSoup(html, "html.parser")
            
            group_info = {
                'group_name_en': group_name.replace('_', ' '),
                'group_name_hangul': self._get_hangul_name(soup),
                'debut_date': self._get_debut_date(soup),
                'entertainment': self._get_entertainment(soup),
                'entertainment_link': self._get_entertainment_link(soup),
                'members': self._get_members(soup),
                'fandom_name': self._get_fandom_name(soup),
                'sns_links': self._get_sns_links(soup)
            }
            
            return group_info
            
        except requests.RequestException as e:
            print(f"네트워크 오류: {e}")
            return {}
        except Exception as e:
            print(f"크롤링 오류: {e}")
            return {}
    
    def _get_hangul_name(self, soup) -> str:
        """한글 그룹명 추출"""
        group_name = soup.select_one('[data-source="hangul"] .pi-data-value')
        return group_name.get_text(strip=True) if group_name else ""
    
    def _get_debut_date(self, soup) -> str:
        """데뷔 날짜 추출"""
        debut_block = soup.select_one('[data-source="debut"] .pi-data-value')
        if debut_block:
            debut_parts = debut_block.get_text("||", strip=True).split("||")
            first_debut = debut_parts[0]
            first_debut_date_str = first_debut.split("(")[0].strip()
            
            try:
                debut_date = datetime.strptime(first_debut_date_str, "%B %d, %Y")
                return debut_date.strftime("%Y-%m-%d")
            except ValueError:
                return first_debut_date_str
        return ""
    
    def _get_entertainment(self, soup) -> str:
        """소속사명 추출 - 첫 번째 엔터테인먼트 회사"""
        label_block = soup.select_one('[data-source="label"] .pi-data-value')
        if not label_block:
            return ""
        
        # 첫 번째 b 태그(국가 라벨) 다음의 첫 번째 a 태그나 텍스트 찾기
        first_b = label_block.find('b')
        if first_b:
            # b 태그 다음의 첫 번째 a 태그 찾기
            a_tag = first_b.find_next('a')
            if a_tag:
                return a_tag.get_text(strip=True)
        
        # b 태그가 없으면 기존 방식
        labels = label_block.get_text("||", strip=True).split("||")
        return labels[0].strip() if labels else ""
    
    def _get_entertainment_link(self, soup) -> str:
        """소속사 링크 추출 - 첫 번째 엔터테인먼트 회사 링크"""
        label_block = soup.select_one('[data-source="label"] .pi-data-value')
        if not label_block:
            return ""
        
        # 첫 번째 b 태그(국가 라벨) 다음의 첫 번째 a 태그 찾기
        first_b = label_block.find('b')
        if first_b:
            a_tag = first_b.find_next('a', href=True)
            if a_tag:
                href = a_tag['href']
                return href if href.startswith('http') else f"{self.base_wiki_url}{href}"
        
        # b 태그가 없으면 기존 방식
        labels = label_block.get_text("||", strip=True).split("||")
        if labels:
            first_label = labels[0].strip()
            a_tag = label_block.find('a', string=lambda text: text and text.strip() == first_label)
            if a_tag and a_tag.has_attr('href'):
                href = a_tag['href']
                return href if href.startswith('http') else f"{self.base_wiki_url}{href}"
        
        return ""
    
    def _get_members(self, soup) -> List[Dict]:
        """멤버 정보 추출"""
        members = []
        
        # 현재 활동 멤버
        members_current = soup.select_one('[data-source="current"] .pi-data-value > ul')
        if members_current:
            for a in members_current.select('a'):
                name = a.get_text(strip=True)
                href = a.get("href")
                link = f"{self.base_wiki_url}{href}"
                members.append({"name": name, "href": link, "status": "current"})
        
        # 비활동 멤버
        members_inactive = soup.select_one('[data-source="inactive"] .pi-data-value > ul')
        if members_inactive:
            for a in members_inactive.select('a'):
                name = a.get_text(strip=True)
                href = a.get("href")
                link = f"{self.base_wiki_url}{href}"
                members.append({"name": name, "href": link, "status": "inactive"})
        
        # 탈퇴 멤버
        members_former = soup.select_one('[data-source="former"] .pi-data-value > ul')
        if members_former:
            for a in members_former.select('a'):
                name = a.get_text(strip=True)
                href = a.get("href")
                link = f"{self.base_wiki_url}{href}"
                members.append({"name": name, "href": link, "status": "former"})
        
        return members
    
    def _get_fandom_name(self, soup) -> str:
        """팬덤명 추출"""
        fandom_block = soup.select_one('[data-source="fandom"] .pi-data-value')
        return fandom_block.get_text(strip=True) if fandom_block else ""
    
    def _get_sns_links(self, soup) -> Dict:
        """SNS 링크 추출"""
        sns_block = soup.select_one('[data-source="sns"]')
        sns_data = {}
        current_country = "KR"
        
        if sns_block:
            for elem in sns_block.children:
                if elem.name == 'b':
                    country_text = elem.get_text(strip=True).rstrip(':')
                    current_country = country_text
                    if current_country not in sns_data:
                        sns_data[current_country] = []
                
                if elem.name == 'span':
                    a = elem.find('a')
                    img = elem.find('img')
                    if a and img:
                        href = a.get('href')
                        platform = img.get('data-image-name', "").replace(" Icon.png", "")
                        if current_country not in sns_data:
                            sns_data[current_country] = []
                        sns_data[current_country].append({"platform": platform, "href": href})
        
        return sns_data
    
    def create_csv_from_groups(self, group_names: List[str], output_filename: str = "kpop_groups_info.csv", group_type: str = "Unknown"):
        """
        여러 그룹의 정보를 크롤링하여 CSV 파일로 저장하고, 멤버 페이지명 리스트도 반환합니다.
        
        Args:
            group_names (List[str]): 크롤링할 그룹명 리스트
            output_filename (str): 출력할 CSV 파일명
            group_type (str): 그룹 타입 (예: "Girl Group", "Boy Group", "Co-ed", "Solo" 등)
        
        Returns:
            tuple: (DataFrame, List[Dict]) - CSV DataFrame과 멤버 페이지명 리스트
        """
        all_data = []
        member_pages = []  # API 호출용 멤버 페이지명 리스트
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # 현재 시간
        
        for group_name in group_names:
            print(f"크롤링 중: {group_name}")
            group_info = self.get_group_info(group_name)
            
            # 크롤링 실패 시에도 null 값으로 데이터 추가
            if not group_info:
                print(f"정보를 가져올 수 없습니다: {group_name}")
                group_info = {
                    'group_name_en': group_name.replace('_', ' '),
                    'group_name_hangul': None,
                    'debut_date': None,
                    'entertainment': None,
                    'entertainment_link': None,
                    'members': [],
                    'fandom_name': None,
                    'sns_links': {}
                }
            
            # 멤버 페이지명 추출 (current, inactive만 포함, former 제외)
            # 중복 없이 추가
            for member in group_info['members']:
                if member['status'] in ['current', 'inactive']:  # former 제외
                    page_name = member['href'].split('/wiki/')[-1] if '/wiki/' in member['href'] else member['name']
                    
                    # 중복 체크
                    if not any(mp['page_name'] == page_name for mp in member_pages):
                        member_pages.append({
                            'group_name': group_name.replace('_', ' '),
                            'page_name': page_name,
                            'member_name': member['name']
                        })
            
            # 멤버를 상태별로 분리
            current_members = []
            inactive_members = []
            former_members = []
            
            for member in group_info['members']:
                if member['status'] == 'current':
                    current_members.append(member['name'])
                elif member['status'] == 'inactive':
                    inactive_members.append(member['name'])
                elif member['status'] == 'former':
                    former_members.append(member['name'])
            
            # 멤버 정보를 문자열로 변환 (null 처리)
            current_members_str = "; ".join(current_members) if current_members else None
            inactive_members_str = "; ".join(inactive_members) if inactive_members else None
            former_members_str = "; ".join(former_members) if former_members else None
            
            # SNS 링크를 문자열로 변환 (null 처리)
            sns_str = json.dumps(group_info['sns_links'], ensure_ascii=False) if group_info['sns_links'] else None
            
            row_data = {
                'group_name_en': group_info['group_name_en'],
                'group_name_kr': group_info['group_name_hangul'] or None,
                'debut_date': group_info['debut_date'] or None,
                'entertainment_name': group_info['entertainment'] or None,
                'entertainment_link': group_info['entertainment_link'] or None,
                'member_current': current_members_str,
                'member_inactive': inactive_members_str,
                'member_former': former_members_str,
                'fandom_name': group_info['fandom_name'] or None,
                'sns': sns_str,
                'group_type': group_type,
                'update_at': current_time
            }
            
            all_data.append(row_data)
        
        # 데이터가 있든 없든 항상 DataFrame 생성
        df = pd.DataFrame(all_data)
        df.to_csv(output_filename, index=False, encoding='utf-8-sig')
        print(f"CSV 파일이 생성되었습니다: {output_filename}")
        print(f"총 {len(member_pages)}명의 멤버 페이지명이 수집되었습니다.")
        
        return df, member_pages

In [5]:
# 크롤러 객체 생성
crawler = KpopGroupCrawler_memberDict()

In [4]:
# girl 그룹 내보내기
girl_groups = ["Red_Velvet", "BLACKPINK", "TWICE", "ITZY", "Aespa", "MAMAMOO", "CrazAngel", "IVE", "ILLIT", "BABYMONSTER", "NJZ"]

df, girl_groups_member_pages = crawler.create_csv_from_groups(girl_groups, "girl_groups_info.csv", "Girl Group")


NameError: name 'crawler' is not defined

In [ ]:
# boy 그룹 내보내기
boy_groups = ["BTS", "Stray_Kids", "SEVENTEEN", "ENHYPEN", "TXT", "NCT", "NCT_127", "ATEEZ", "EXO", "P1Harmony", "SF9", "VICTON", "CRAVITY"]
# 유닛 포함해서 그룹 내 멤버 중복되게 내보내기

df, boy_groups_member_pages = crawler.create_csv_from_groups(boy_groups, "boy_groups_info.csv", "Boy Group")

크롤링 중: BTS
크롤링 중: Stray_Kids
크롤링 중: SEVENTEEN
크롤링 중: ENHYPEN
크롤링 중: TXT
크롤링 중: NCT
크롤링 중: NCT_127
크롤링 중: ATEEZ
크롤링 중: EXO
크롤링 중: P1Harmony
크롤링 중: SF9
크롤링 중: VICTON
크롤링 중: CRAVITY
CSV 파일이 생성되었습니다: girl_groups_info.csv
총 111명의 멤버 페이지명이 수집되었습니다.


In [12]:
# NCT 멤버 중복되었는지 확인

df_duplicated_check = pd.DataFrame(boy_groups_member_pages)
print(df_duplicated_check.duplicated(subset=['page_name']).sum())  # 중복 개수
print(df_duplicated_check[df_duplicated_check.duplicated(subset=['page_name'], keep=False)])  # 중복된 행 보기

0
Empty DataFrame
Columns: [group_name, page_name, member_name]
Index: []


In [17]:
# 설정값
base_url = "https://kpop.fandom.com/api.php"
base_wiki_url = "https://kpop.fandom.com"

def get_birth_name(soup):
    """본명 추출"""
    selectors = [
        '[data-source="birth_name"] .pi-data-value',
        '[data-source="birthname"] .pi-data-value',
        '[data-source="real_name"] .pi-data-value',
        '[data-source="realname"] .pi-data-value'
    ]
    
    for selector in selectors:
        birth_name = soup.select_one(selector)
        if birth_name:
            return birth_name.get_text(strip=True)
    return ""

def get_birth_date_artist(soup):
    """생년월일 추출"""
    birth_block = soup.select_one('[data-source="birth_date"] .pi-data-value')
    if birth_block:
        birth_text = birth_block.get_text(strip=True)
        birth_date_str = birth_text.split("(")[0].strip()
        
        try:
            birth_date = datetime.strptime(birth_date_str, "%B %d, %Y")
            return birth_date.strftime("%Y-%m-%d")
        except ValueError:
            return birth_date_str
    return ""

def _get_entertainment(soup) -> str:
    """소속사명 추출 - 첫 번째 엔터테인먼트 회사"""
    label_block = soup.select_one('[data-source="agency"] .pi-data-value')
    if not label_block:
        return ""
        
    # 첫 번째 b 태그(국가 라벨) 다음의 첫 번째 a 태그나 텍스트 찾기
    first_b = label_block.find('b')
    if first_b:
        # b 태그 다음의 첫 번째 a 태그 찾기
        a_tag = first_b.find_next('a')
        if a_tag:
            return a_tag.get_text(strip=True)
        
    # b 태그가 없으면 기존 방식
    labels = label_block.get_text("||", strip=True).split("||")
    return labels[0].strip() if labels else ""
    
def _get_entertainment_link(soup) -> str:
    """소속사 링크 추출 - 첫 번째 엔터테인먼트 회사 링크"""
    label_block = soup.select_one('[data-source="agency"] .pi-data-value')
    if not label_block:
        return ""
        
    # 첫 번째 b 태그(국가 라벨) 다음의 첫 번째 a 태그 찾기
    first_b = label_block.find('b')
    if first_b:
        a_tag = first_b.find_next('a', href=True)
        if a_tag:
            href = a_tag['href']
            return href if href.startswith('http') else f"{base_wiki_url}{href}"
        
    # b 태그가 없으면 기존 방식
    labels = label_block.get_text("||", strip=True).split("||")
    if labels:
        first_label = labels[0].strip()
        a_tag = label_block.find('a', string=lambda text: text and text.strip() == first_label)
        if a_tag and a_tag.has_attr('href'):
            href = a_tag['href']
            return href if href.startswith('http') else f"{base_wiki_url}{href}"
        
    return ""
#

def create_artist_csv(member_pages, output_filename="artists_info.csv"):
    """
    멤버 페이지 리스트를 받아서 개별 아티스트 정보를 크롤링하여 CSV로 저장합니다.
    
    Args:
        member_pages (List[Dict]): [{'group_name': '그룹명', 'page_name': '페이지명'}, ...] 형태의 리스트
        output_filename (str): 출력할 CSV 파일명
    
    Returns:
        DataFrame: 아티스트 정보가 담긴 DataFrame
    """
    all_artist_data = []
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    for member_info in member_pages:
        group_name = member_info['group_name']
        page_name = member_info['page_name']
        member_name = member_info['member_name']
        
        print(f"크롤링 중: {group_name} - {page_name}")
        
        try:
            params = {
                "action": "parse",
                "page": page_name,
                "format": "json"
            }
            
            res = requests.get(base_url, params=params)
            res.raise_for_status()
            data = res.json()
            
            if 'parse' not in data:
                try:
                    # API 호출용으로 디코딩
                    decoded_page_name = unquote(page_name)
                    
                    params = {
                        "action": "parse",
                        "page": decoded_page_name,
                        "format": "json"
                    }
                    res = requests.get(base_url, params=params)
                    res.raise_for_status()
                    data = res.json()
                except Exception as e:
                    print(f"페이지를 찾을 수 없습니다: {page_name}")
                    row_data = {
                        'group_name': group_name,
                        'page_name': page_name,
                        'member_name': member_name,
                        'birth_name': None,
                        'birth_date': None,
                        'agency_name': None,
                        'agency_href': None,
                        'update_at': current_time
                    }
                    all_artist_data.append(row_data)
                    continue
            
            html = data["parse"]["text"]["*"]
            soup = BeautifulSoup(html, "html.parser")
            
            # 정보 추출
            birth_name = get_birth_name(soup)
            birth_date = get_birth_date_artist(soup)
            agency_name = _get_entertainment(soup)
            agency_href = _get_entertainment_link(soup)
            
            row_data = {
                'group_name': group_name,
                'page_name': page_name,
                'member_name': member_name,
                'birth_name': birth_name or None,
                'birth_date': birth_date or None,
                'agency_name': agency_name or None,
                'agency_href': agency_href or None,
                'update_at': current_time
            }
            
            all_artist_data.append(row_data)
            
        except Exception as e:
            print(f"오류 ({page_name}): {e}")
            row_data = {
                'group_name': group_name,
                'page_name': page_name,
                'member_name': member_name,
                'birth_name': None,
                'birth_date': None,
                'agency_name': None,
                'agency_href': None,
                'update_at': current_time
            }
            all_artist_data.append(row_data)
    
    # CSV 저장
    df = pd.DataFrame(all_artist_data)
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"Artist CSV 파일이 생성되었습니다: {output_filename}")
    print(f"총 {len(all_artist_data)}명의 아티스트 정보가 수집되었습니다.")
    
    return df

In [18]:
artist_df = create_artist_csv(boy_groups_member_pages, "boy_groups_artists.csv")

크롤링 중: BTS - Jin_(BTS)
크롤링 중: BTS - Suga
크롤링 중: BTS - J-Hope
크롤링 중: BTS - RM
크롤링 중: BTS - Jimin_(BTS)
크롤링 중: BTS - V_(BTS)
크롤링 중: BTS - Jung_Kook
크롤링 중: Stray Kids - Bang_Chan
크롤링 중: Stray Kids - Lee_Know
크롤링 중: Stray Kids - Changbin_(Stray_Kids)
크롤링 중: Stray Kids - Hyunjin_(Stray_Kids)
크롤링 중: Stray Kids - Han_(Stray_Kids)
크롤링 중: Stray Kids - Felix
크롤링 중: Stray Kids - Seungmin_(Stray_Kids)
크롤링 중: Stray Kids - I.N_(Stray_Kids)
크롤링 중: SEVENTEEN - S.Coups
크롤링 중: SEVENTEEN - Joshua
크롤링 중: SEVENTEEN - Jun_(SEVENTEEN)
크롤링 중: SEVENTEEN - DK_(SEVENTEEN)
크롤링 중: SEVENTEEN - Mingyu_(SEVENTEEN)
크롤링 중: SEVENTEEN - The8
크롤링 중: SEVENTEEN - Seungkwan
크롤링 중: SEVENTEEN - Vernon
크롤링 중: SEVENTEEN - Dino_(SEVENTEEN)
크롤링 중: SEVENTEEN - Jeonghan
크롤링 중: SEVENTEEN - Hoshi
크롤링 중: SEVENTEEN - Wonwoo_(SEVENTEEN)
크롤링 중: SEVENTEEN - Woozi
크롤링 중: ENHYPEN - Heeseung
크롤링 중: ENHYPEN - Jay_(ENHYPEN)
크롤링 중: ENHYPEN - Jake_(ENHYPEN)
크롤링 중: ENHYPEN - Sunghoon_(ENHYPEN)
크롤링 중: ENHYPEN - Sunoo
크롤링 중: ENHYPEN - Jungwon_(ENHYP